## YOLOV5 FINE TUNING

### COCO DATASET

In [3]:
import subprocess
import re
import time
import os
from tqdm import tqdm

# Set paths
DATASET_YAML = "dataset.yaml"  # Update if needed
MODEL_PATH = "yolov5nu.pt"  # Pre-trained YOLOv5 model
EPOCHS = 100  # Training epochs
BATCH_SIZE = 16  # Adjust based on GPU memory
IMG_SIZE = 640  # Image size
DEVICE = "cuda"  # Use "cpu" if no GPU
WORKERS = str(min(4, max(1, os.cpu_count() // 2)))  # Auto-set workers

# Training command
command = [
    "python", "yolov5/train.py",
    "--img", str(IMG_SIZE),
    "--batch", str(BATCH_SIZE),
    "--epochs", str(EPOCHS),
    "--data", DATASET_YAML,
    "--weights", MODEL_PATH,
    "--device", DEVICE,
    "--workers", WORKERS
]

# Initialize tqdm progress bar
pbar = tqdm(total=EPOCHS, desc="Training Progress", unit="epoch", dynamic_ncols=True)

# Start training process
start_time = time.time()
with subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1) as process:
    for line in process.stdout:
        print(line, end="")  # Print training logs in real-time

        # Match YOLOv5 epoch progress
        match = re.search(r"Epoch\s+(\d+)/(\d+)", line)
        if match:
            current_epoch = int(match.group(1))
            total_epochs = int(match.group(2))

            # Update tqdm progress bar
            pbar.n = current_epoch
            elapsed_time = time.time() - start_time
            avg_time_per_epoch = elapsed_time / max(1, current_epoch)
            remaining_time = avg_time_per_epoch * (total_epochs - current_epoch)

            pbar.set_description(f"Training Progress ⏳ {remaining_time:.2f} sec remaining")
            pbar.update(1)  # Move progress forward

pbar.close()
process.wait()  # Ensure process finishes
print("\n✅ Training completed!")


Training Progress:   0%|          | 0/100 [00:00<?, ?epoch/s]

Error decoding JSON from C:\Users\Kirk Recio\AppData\Roaming\Ultralytics\persistent_cache.json. Starting with an empty dictionary.
train: weights=yolov5nu.pt, cfg=, data=dataset.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5\data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=cuda, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=4, project=yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 5 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
fatal: cannot change to 'C:\Users\Kirk':

Training Progress:   0%|          | 0/100 [00:15<?, ?epoch/s]


✅ Training completed!


In [24]:
%ls

 Volume in drive C is Acer
 Volume Serial Number is 8460-89F9

 Directory of c:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Training\yolov5

03/24/2025  03:01 PM    <DIR>          .
03/24/2025  05:51 PM    <DIR>          ..
03/24/2025  01:20 PM             3,923 .dockerignore
03/24/2025  01:20 PM                77 .gitattributes
03/24/2025  01:20 PM    <DIR>          .github
03/24/2025  01:20 PM             4,268 .gitignore
03/24/2025  01:27 PM    <DIR>          __pycache__
03/24/2025  01:20 PM            14,440 benchmarks.py
03/24/2025  01:20 PM               407 CITATION.cff
03/24/2025  01:20 PM    <DIR>          classify
03/24/2025  01:20 PM             5,082 CONTRIBUTING.md
03/24/2025  01:20 PM    <DIR>          data
03/24/2025  01:20 PM            24,199 detect.py
03/24/2025  01:20 PM            70,122 export.py
03/24/2025  01:20 PM            24,504 hubconf.py
03/24/2025  01:20 PM            35,184 LICENSE
03/24/2025  01:27 PM    <DIR>          models
03/24/20

### COCO DATASET 2 

In [20]:
import yaml
print(yaml.safe_load(open("dataset.yaml")))


{'train': '../cocosubset/train2017', 'val': '../cocosubset/val2017', 'nc': 80, 'names': ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'TV', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']}


### ALMOST WORKING 

In [13]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'TensorBoard' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

Reusing TensorBoard on port 6006 (pid 16720), started 3 days, 0:27:38 ago. (Use '!kill 16720' to kill it.)

In [2]:
%cd yolov5/

!python train.py --img 640 --batch 1 --epochs 1 --data dataset.yaml --weights yolov5n.pt    



c:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Training\yolov5


C:\Users\Kirk Recio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


train: weights=yolov5n.pt, cfg=, data=dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=1, batch_size=1, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github:  YOLOv5 is out of date by 4 commits. Use 'git pull' or 'git clone https://github.com/ultralytics/yolov5' to update.
fatal: cannot change to 'C:\Users\Kirk': Invalid argument
YOLOv5  2025-3-25 Python-3.12.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

hyperparameters: lr0=0.01, lrf=0.01, m

In [15]:
cache_file = r"C:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Training\cocosubset\train2017.cache"

if os.path.exists(cache_file):
    os.remove(cache_file)
    print("✅ Deleted old cache file!")
else:
    print("❌ Cache file not found!")


✅ Deleted old cache file!


In [1]:
#@title Select YOLOv5 🚀 logger {run: 'auto'}
logger = 'Tensorboard' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

In [1]:

# !python train.py --data ../cocosubset/dataset.yaml --weights yolov5n.pt --epochs 1 --batch-size 16 --cache
%cd ..
%cd yolov5
# !python train.py --img 640 --batch 1 --epochs 1 dataset.yaml --weights yolov5n.pt --cache # Almost working 

# !python train.py --img 640 --batch 1 --epochs 1 --data dataset.yaml --weights yolov5n.pt --cache
# !python train.py --img 640 --batch 1 --epochs 1 --data dataset.yaml --weights yolov5n.pt --cache

!python train.py --img 640 --batch 1 --epochs 1 --data dataset.yaml --weights yolov5n.pt --cache

# !python train.py --img 640 --batch 1 --epochs 1 --data dataset.yaml --weights yolov5n.pt --cache


c:\Users\Kirk Recio\Documents\PYTHON\Project Testing
[WinError 2] The system cannot find the file specified: 'yolov5'
c:\Users\Kirk Recio\Documents\PYTHON\Project Testing


C:\Users\Kirk Recio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
C:\Users\Kirk Recio\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
python: can't open file 'c:\\Users\\Kirk Recio\\Documents\\PYTHON\\Project Testing\\train.py': [Errno 2] No such file or directory


#### END OF ALMOST WORKING

In [ ]:
%cd yolov5
!python train.py --img 640 --batch 16 --epochs 50 --data ../dataset.yaml --weights yolov5nu.pt



[WinError 2] The system cannot find the file specified: 'yolov5'
c:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Training\yolov5


train: weights=yolov5nu.pt, cfg=, data=../dataset.yaml, hyp=data\hyps\hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data\hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 
Traceback (most recent call last):
  File "c:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Training\yolov5\train.py", line 986, in <module>
    main(opt)
  File "c:\Users\Kirk Recio\Documents\PYTHON\Project Testing\Data Cleaning Trainin

In [1]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.cuda.get_device_name(0))  # Should print your GPU name


True
NVIDIA GeForce RTX 3050 Laptop GPU


### POTHOLE DATASET

In [2]:
import os
import time

# Start timing
start_time = time.time()

# Run YOLOv5 training
os.system("python yolov5/train.py --weights yolov5s.pt --data pothole.yaml --epochs 100 --batch-size 16 --save-period 10")

# Calculate elapsed time
elapsed_time = time.time() - start_time
print(f"\n🚀 Training completed in {elapsed_time:.2f} seconds ({elapsed_time / 60:.2f} minutes)")



🚀 Training completed in 20.39 seconds (0.34 minutes)


## EASY OCR 

### ICDAR 2015 DATASET